## Q1. Notebook

In [ ]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.1


In [ ]:
!python -V

Python 3.9.19


In [1]:
import pickle
import pandas as pd

In [2]:
year = 2023
month = 3

In [3]:
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)


/home/codespace/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/envs/mlops/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
y_pred.std()

6.247488852238703

#### Answer: standard deviation of the predicted duration for this dataset is
            6.247488852238703

## Q2. Preparing the output

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [15]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [16]:
!ls -lh output

total 66M
-rw-rw-rw- 1 codespace codespace 66M Jul 21 23:31 yellow_tripdata_2023-03.parquet


#### Answer: output file has size 
            66M

## Q3. Creating the scoring script

#### Answer: 
##### jupyter nbconvert --to script starter.ipynb

## Q4. Virtual environment

#### Answer:  first hash for the Scikit-Learn dependency is
                "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"

![alt text](image-1.png)

## Q5. Parametrize the script
#### Answer:  mean predicted duration 14.29

![alt text](image-2.png)

## Q6. Docker container
#### Answer: mean predicted duration for May 2023
            0.19174419265916945

![alt text](image-4.png)